# Dataset

> a class based approach for interacting with Domo Datasets


In [ ]:
# | default_exp classes.DomoDataset

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
# |export
from fastcore.basics import patch

In [ ]:
# | exporti
from dataclasses import dataclass, field
from typing import Any, List, Optional

import json

import aiohttp

# import datetime as dt

# import asyncio
# import importlib
# import io
# import json
# from enum import Enum, auto
# from pprint import pprint

# import pandas as pd

# from ..utils import Exceptions as ex
# from ..utils.Base import Base
# from ..utils.chunk_execution import chunk_list
# from . import DomoCertification as dmdc
# from . import DomoPDP as dmpdp
# from . import DomoTag as dmtg


import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.dataset as dataset_routes

## Component Classes

The `DomoDataset_Schema` class will be a subclass of `DomoDataset`. It will handle all of the methods for interacting with schemas.

- In execution, the schema is separate from the data that gets uploaded from Vault to Adrenaline. The domo schema defines how the data is loaded into Vault.
- Be cognizant to match dataset uploads with schema definitions. If the schema and uploaded data types do not match, the dataset may be unable to index in Adrenaline (and therefore not update).


In [ ]:
# | export


class DatasetSchema_AuthNotProvidedError(Exception):
    """return if DatasetSchema request cannot access an auth object"""

    def __init__(self, dataset_id):
        message = f"valid Auth object not provided to dataset - {dataset_id}"
        super().__init__(message)


class DatasetSchema_DatasetNotProvidedError(Exception):
    """return if DatasetSchema request does not have a dataset id"""

    def __init__(self):
        message = f"dataset_id not provided"
        super().__init__(message)


@dataclass
class DomoDataset_Schema_Column:
    name: str
    id: str
    type: str

    @classmethod
    def _from_json(cls, json_obj):
        dd = util_dd.DictDot(json_obj)
        return cls(name=dd.name, id=dd.id, type=dd.type)


@dataclass
class DomoDataset_Schema:
    """class for interacting with dataset schemas"""

    dataset: any = None
    columns: List[DomoDataset_Schema_Column] = field(default_factory=list)

    async def get(
        self,
        auth: Optional[dmda.DomoAuth] = None,
        dataset_id: str = None,
        debug_api: bool = False,
        return_raw_res: bool = False,  # return the raw response
    ) -> List[DomoDataset_Schema_Column]:

        """method that retrieves schema for a dataset"""

        if self.dataset and (not self.dataset.auth and not auth):
            raise DatasetSchema_AuthNotProvidedError(dataset_id = self.dataset.id)

        auth = auth or self.dataset.auth

        if not self.dataset and not dataset_id:
            raise DatasetSchema_DatasetNotProvidedError()

        dataset_id = dataset_id or self.dataset.id

        res = await dataset_routes.get_schema(
            auth=auth, dataset_id=dataset_id, debug_api=debug_api
        )

        if return_raw_res:
            return res.response

        if res.status == 200:
            json_list = res.response.get("tables")[0].get("columns")

            self.columns = [
                DomoDataset_Schema_Column._from_json(json_obj=json_obj)
                for json_obj in json_list
            ]

            return self.columns

In [ ]:
show_doc(DomoDataset_Schema.get)

---

[source](https://github.com/jaewilson07/domo_library/blob/main/domolibrary/classes/DomoDataset.py#L78){target="_blank" style="float:right; font-size:smaller"}

### DomoDataset_Schema.get

>      DomoDataset_Schema.get
>                              (auth:Optional[domolibrary.client.DomoAuth.DomoAu
>                              th]=None, dataset_id:str=None,
>                              debug_api:bool=False, return_raw_res:bool=False)

method that retrieves schema for a dataset

#### Sample implementation of getting a dataset schema

Standard implementation will be to access the `DomoDataset_Schema` class as the `DomoDataset.schema` property


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

ds_schema = DomoDataset_Schema()

await ds_schema.get(auth=token_auth, dataset_id=os.environ["DOJO_DATASET_ID"])


[DomoDataset_Schema_Column(name='objectID', id='objectID', type='STRING'),
 DomoDataset_Schema_Column(name='url', id='url', type='STRING'),
 DomoDataset_Schema_Column(name='Title', id='Title', type='STRING'),
 DomoDataset_Schema_Column(name='article', id='article', type='STRING'),
 DomoDataset_Schema_Column(name='views', id='views', type='LONG'),
 DomoDataset_Schema_Column(name='created_dt', id='created_dt', type='DATETIME'),
 DomoDataset_Schema_Column(name='published_dt', id='published_dt', type='DATETIME')]

In [ ]:
# | export

class DatasetTags_AuthNotProvidedError(Exception):
    """return if DatasetTags request cannot access an auth object"""

    def __init__(self, id):
        message = f"valid Auth object not provided to dataset - {id}"
        super().__init__(message)


class DatasetTags_SetTagsError(Exception):
    """return if DatasetTags request is not successfull"""

    def __init__(self, dataset_id, domo_instance):
        message = f"failed to set tags on dataset - {dataset_id} in {domo_instance}"
        super().__init__(message)


@dataclass
class DomoDataset_Tags:
    """class for interacting with dataset tags"""

    dataset: any = None
    tag_ls: List[str] = field(default_factory=list)

    def _have_prereqs(self, auth, dataset_id):
        """tests if have a parent dataset or prerequsite dataset_id and auth object"""

        if self.dataset and (not self.dataset.auth and not auth):
            raise DatasetTags_AuthNotProvidedError(self.dataset.id)

        auth = auth or self.dataset.auth

        if not self.dataset and not auth:
            raise DatasetTags_AuthNotProvidedError(self.dataset.id)

        dataset_id = dataset_id or self.dataset.id

        return auth, dataset_id

    async def get(
        self,
        dataset_id: str = None,
        auth: Optional[dmda.DomoAuth] = None,
        debug_api: bool = False,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> List[str]:  # returns a list of tags
        """gets the existing list of dataset_tags"""

        auth, dataset_id = self._have_prereqs(auth=auth, dataset_id=dataset_id)

        res = await dataset_routes.get_dataset_by_id(
            dataset_id=dataset_id, auth=auth, debug_api=debug_api, session=session
        )

        if res.is_success == False:
            print(res)
            return None

        if res.is_success == True:
            tag_ls = json.loads(res.response.get("tags"))
            self.tag_ls = tag_ls

            return tag_ls

    async def set(
        self,
        tag_ls: [str],
        dataset_id: str = None,
        auth: Optional[dmda.DomoAuth] = None,
        debug_api: bool = False,
        session: Optional[aiohttp.ClientSession] = None,
    ) -> List[str]: # returns a list of tags
        """replaces all tags with a new list of dataset_tags"""

        auth, dataset_id = self._have_prereqs(auth=auth, dataset_id=dataset_id)

        res = await dataset_routes.set_dataset_tags(
            auth=auth,
            tag_ls=list(set(tag_ls)),
            dataset_id=dataset_id,
            debug_api=debug_api,
            session=session,
        )

        if res.status != 200:
            raise DatasetTags_SetTagsError(
                dataset_id=dataset_id, domo_instance=auth.domo_instance
            )

        await self.get(dataset_id=dataset_id, auth=auth)

        return self.tag_ls

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

ds_tag = DomoDataset_Tags()
await ds_tag.get(auth=token_auth, dataset_id=os.environ["DOJO_DATASET_ID"])


['developer_documentation', 'Jan-11-2023 12:50', 'hackercore']

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

today = dt.datetime.now().strftime("%b-%d-%Y %H:%M")
ds_tag = DomoDataset_Tags()
await ds_tag.set(
    auth=token_auth,
    dataset_id=os.environ["DOJO_DATASET_ID"],
    tag_ls=["developer_documentation", "hackercore", today],
)

['developer_documentation', 'hackercore', 'Jan-11-2023 12:50']

In [ ]:
#| export

@patch
async def add(
    self: DomoDataset_Tags,
    add_tag_ls: [str],
    dataset_id: str = None,
    auth: Optional[dmda.DomoAuth] = None,
    debug_api: bool = False,
    session: Optional[aiohttp.ClientSession] = None,
) -> List[str]:  # returns a list of tags
    """appends tags to the list of existing dataset_tags"""

    auth, dataset_id = self._have_prereqs(auth=auth, dataset_id=dataset_id)

    existing_tag_ls = await self.get(dataset_id=dataset_id, auth=auth)
    add_tag_ls += existing_tag_ls

    return await self.set(
        auth=auth,
        dataset_id=dataset_id,
        tag_ls=list(set(add_tag_ls)),
        debug_api=debug_api,
        session=session,
    )

In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

today_year = dt.datetime.today().strftime("%Y")
ds_tag = DomoDataset_Tags()
await ds_tag.add(
    auth=token_auth, dataset_id=os.environ["DOJO_DATASET_ID"], add_tag_ls=[today_year]
)

['2023', 'developer_documentation', 'hackercore', 'Jan-11-2023 12:50']

In [ ]:
#| export
@patch
async def remove(self: DomoDataset_Tags,
                 remove_tag_ls: [str],
                 dataset_id: str = None,
                 auth: dmda.DomoFullAuth = None,
                 debug_api: bool = False,
                 session: Optional[aiohttp.ClientSession] = None
                 ) -> List[str]:  # returns a list of tags
    """removes tags from the existing list of dataset_tags"""

    auth, dataset_id = self._have_prereqs(auth=auth, dataset_id=dataset_id)

    existing_tag_ls = await self.get(dataset_id=dataset_id, auth=auth)

    existing_tag_ls = [
        ex for ex in existing_tag_ls if ex not in remove_tag_ls]

    return await self.set(auth=auth,
                          dataset_id=dataset_id,
                          tag_ls=list(set(existing_tag_ls)),
                          debug_api=debug_api, session=session)


In [ ]:
import os
import datetime as dt

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

today_year = dt.datetime.today().strftime("%Y")

ds_tag = DomoDataset_Tags()

await ds_tag.remove(
    auth=token_auth, dataset_id=os.environ["DOJO_DATASET_ID"], remove_tag_ls=[ today_year])

['developer_documentation', 'hackercore', 'Jan-11-2023 12:50']

# MAIN - Domo Dataset


In [ ]:
# | export
@dataclass
class DomoDataset:
    "interacts with domo datasets"

    auth: dmda.DomoAuth = field(repr=False, default=None)

    id: str = ""
    display_type: str = ""
    data_provider_type: str = ""
    name: str = ""
    description: str = ""
    row_count: int = None
    column_count: int = None

    stream_id: int = None

    owner: dict = field(default_factory=dict)
    formula: dict = field(default_factory=dict)

    schema: DomoDataset_Schema = field(default=None)
    # tags: Dataset_Tags = field(default = None)

    # certification: dmdc.DomoCertification = None
    # PDPPolicies: dmpdp.Dataset_PDP_Policies = None

    def __post_init__(self):
        self.schema = DomoDataset_Schema(dataset=self)
        # self.tags = Dataset_Tags(dataset=self)

        # self.PDPPolicies = dmpdp.Dataset_PDP_Policies(self)

    def display_url(self):
        return f"https://{self.auth.domo_instance }.domo.com/datasources/{self.id}/details/overview"

#### sample class-based implementation of get schema.


In [ ]:
# this sample returns raw response from the api

import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

ds = DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])

raw_res = await ds.schema.get(return_raw_res=True)

pd.DataFrame(raw_res.get("tables")[0].get("columns"))


,name,id,type,visible,order
0,objectID,objectID,STRING,True,0
1,url,url,STRING,True,0
2,Title,Title,STRING,True,0
3,article,article,STRING,True,0
4,views,views,LONG,True,0
5,created_dt,created_dt,DATETIME,True,0
6,published_dt,published_dt,DATETIME,True,0


In [ ]:
# this sample returns class-based response from the api
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

ds = DomoDataset(auth=token_auth, id=os.environ["DOJO_DATASET_ID"])

await ds.schema.get()


[DomoDataset_Schema_Column(name='objectID', id='objectID', type='STRING'),
 DomoDataset_Schema_Column(name='url', id='url', type='STRING'),
 DomoDataset_Schema_Column(name='Title', id='Title', type='STRING'),
 DomoDataset_Schema_Column(name='article', id='article', type='STRING'),
 DomoDataset_Schema_Column(name='views', id='views', type='LONG'),
 DomoDataset_Schema_Column(name='created_dt', id='created_dt', type='DATETIME'),
 DomoDataset_Schema_Column(name='published_dt', id='published_dt', type='DATETIME')]

In [ ]:
# | export


@patch(cls_method=True)
async def get_from_id(
    cls: DomoDataset,
    dataset_id: str,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw_res: bool = False,
):

    """retrieves dataset metadata"""

    res = await dataset_routes.get_dataset_by_id(
        auth=auth, dataset_id=dataset_id, debug_api=debug_api
    )

    if return_raw_res:
        return res.response

    dd = util_dd.DictDot(res.response)
    ds = cls(
        auth=auth,
        id=dd.id,
        display_type=dd.displayType,
        data_provider_type=dd.dataProviderType,
        name=dd.name,
        description=dd.description,
        owner=dd.owner,
        formula=dd.properties.formulas.formulas,
        stream_id=dd.streamId,
        row_count=int(dd.rowCount),
        column_count=int(dd.columnCount),
    )

    # if dd.tags:
    #     ds.tags.tag_ls = json.loads(dd.tags)

    # if dd.certification:
    #     # print('class def certification', dd.certification)
    #     ds.certification = dmdc.DomoCertification._from_json(
    #         dd.certification)

    return ds


#### sample implementation of get_from_id


In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

try:
    await DomoDataset.get_from_id(auth=token_auth, dataset_id="123")
except Exception as e:
    print(e)

dataset - 123 not found in domo-dojo


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoDataset.get_from_id(auth=token_auth, dataset_id=os.environ["DOJO_DATASET_ID"])


DomoDataset(id='04c1574e-c8be-4721-9846-c6ffa491144b', display_type='domo-jupyterdata', data_provider_type='domo-jupyterdata', name='domo_kbs', description=None, row_count=1185, column_count=7, stream_id=825, owner=DictDot(id='1893952720', name='Jae Wilson', type='USER', group=False), formula=DictDot(), schema=DomoDataset_Schema(dataset=..., columns=[]))

In [ ]:
#     @classmethod
#     async def query_dataset(cls,
#                             sql: str,
#                             dataset_id: str,
#                             dev_auth: DomoDeveloperAuth,
#                             debug: bool = False,
#                             session: aiohttp.ClientSession = None) -> pd.DataFrame:

#         if debug:
#             print("query dataset class method")
#             print({'dataset_id': dataset_id,
#                    'dev_auth': dev_auth})

#         res = await dataset_routes.query_dataset_public(dev_auth=dev_auth, id=dataset_id, sql=sql, session=session,
#                                                         debug=debug)

#         if debug:
#             print(res.response)

#         if res.status == 200:
#             df = pd.DataFrame(data=res.response.get('rows'),
#                               columns=res.response.get('columns'))
#             return df
#         return None

#     @classmethod
#     async def query_dataset_private(cls,
#                                     sql: str,
#                                     dataset_id: str,
#                                     full_auth: DomoFullAuth,
#                                     debug: bool = False,
#                                     session: aiohttp.ClientSession = None) -> pd.DataFrame:

#         if debug:
#             print("query dataset class method")
#             print({'dataset_id': dataset_id,
#                    'full_auth': full_auth})

#         res = await dataset_routes.query_dataset_private(full_auth=full_auth, id=dataset_id, sql=sql, session=session,
#                                                          debug=debug)

#         return pd.DataFrame(res)

#     async def upload_csv(self,
#                          upload_df: pd.DataFrame = None,
#                          upload_df_list: list[pd.DataFrame] = None,
#                          upload_file: io.TextIOWrapper = None,

#                          full_auth: DomoFullAuth = None,
#                          upload_method: str = 'REPLACE',
#                          dataset_id: str = None,
#                          dataset_upload_id=None,
#                          partition_key: str = None,
#                          is_index: bool = True,
#                          session: aiohttp.ClientSession = None,
#                          debug: bool = False):

#         full_auth = full_auth or self.full_auth
#         dataset_id = dataset_id or self.id

#         upload_df_list = upload_df_list or [upload_df]

#         # stage 1 get uploadId
#         if not dataset_upload_id:
#             if debug:
#                 print(f"\n\n🎭 starting Stage 1")

#             res = await dataset_routes.upload_dataset_stage_1(full_auth=full_auth,
#                                                               dataset_id=dataset_id,
#                                                               session=session,
#                                                               data_tag=partition_key,
#                                                               debug=debug
#                                                               )
#             if debug:
#                 print(f"\n\n🎭 Stage 1 response -- {res.status}")
#                 print(res)

#             dataset_upload_id = res.response.get('uploadId')

#         # stage 2 upload_dataset

#         if debug:
#             print(
#                 f"\n\n🎭 starting Stage 2 - {len(upload_df_list)} - number of parts")

#         stage_2_res = None

#         if upload_file:
#             if debug:
#                 print('stage 2 - file')
#             stage_2_res = await dataset_routes.upload_dataset_stage_2_file(full_auth=full_auth,
#                                                                            dataset_id=dataset_id,
#                                                                            upload_id=dataset_upload_id,
#                                                                            part_id=1,
#                                                                            file=upload_file,
#                                                                            session=session, debug=debug)
#             if debug:
#                 print(f"🎭 Stage 2 response -- {stage_2_res.status}")
#                 print(stage_2_res.print(is_pretty=True))

#         else:
#             if debug:
#                 print('stage 2 - df')
#             stage_2_res = await asyncio.gather(*[dataset_routes.upload_dataset_stage_2_df(full_auth=full_auth,
#                                                                                           dataset_id=dataset_id,
#                                                                                           upload_id=dataset_upload_id,
#                                                                                           part_id=index + 1,
#                                                                                           upload_df=df,
#                                                                                           session=session, debug=debug) for index, df in enumerate(upload_df_list)])

#             if debug:
#                 for res in stage_2_res:
#                     print(f"🎭 Stage 2 response -- {res.status}")
#                     res.print(is_pretty=True)

#         # return stage_2_res

# #         # stage 3 commit_data
#         if debug:
#             print(f"\n\n🎭 starting Stage 3")
#         await asyncio.sleep(10)

#         stage3_res = await dataset_routes.upload_dataset_stage_3(full_auth=full_auth,
#                                                                  dataset_id=dataset_id,
#                                                                  upload_id=dataset_upload_id,
#                                                                  update_method=upload_method,
#                                                                  data_tag=partition_key,
#                                                                  is_index=False,
#                                                                  session=session,
#                                                                  debug=debug)

#         if debug:
#             print(f"\n🎭 stage 3 res - {res.status}")
#             print(stage3_res)

#         if is_index:
#             await self.index_dataset(full_auth=full_auth,
#                                      dataset_id=dataset_id,
#                                      debug=debug,
#                                      session=session)

#         return stage3_res

#     async def index_dataset(self,
#                             full_auth: DomoFullAuth = None,
#                             dataset_id: str = None,
#                             debug: bool = False,
#                             session: aiohttp.ClientSession = None
#                             ):

#         full_auth = full_auth or self.full_auth
#         dataset_id = dataset_id or self.id
#         return await dataset_routes.index_dataset(full_auth=full_auth, dataset_id=dataset_id, debug=debug,
#                                                   session=session)

#     async def list_partitions(self,
#                               full_auth: DomoFullAuth = None,
#                               dataset_id: str = None,
#                               debug: bool = False,
#                               session: aiohttp.ClientSession = None
#                               ):

#         full_auth = full_auth or self.full_auth
#         dataset_id = dataset_id or self.id

#         res = await dataset_routes.list_partitions(full_auth=full_auth, dataset_id=dataset_id, debug=debug,
#                                                    session=session)
#         if res.status != 200:
#             return None
#         return res.response

#     async def delete_partition(self,
#                                dataset_partition_id: str,

#                                dataset_id: str = None,
#                                empty_df: pd.DataFrame = None,

#                                full_auth: DomoFullAuth = None,

#                                is_index: bool = True,
#                                debug: bool = False,
#                                session: aiohttp.ClientSession = None):

#         is_close_session = True if not session else False

#         session = session or aiohttp.ClientSession()
#         full_auth = full_auth or self.full_auth
#         dataset_id = dataset_id or self.id

# #        if empty_df is None:
# #            empty_df = await self.query_dataset_private(full_auth=full_auth,
# #                                                        dataset_id=dataset_id,
# #                                                        sql="SELECT * from table limit 1",
# #                                                        debug=False)
# #
# #        await self.upload_csv(upload_df=empty_df.head(0),
# #                              upload_method='REPLACE',
# #                              is_index=is_index,
# #                              partition_key=dataset_partition_id,
# #                              session=session,
# #                              debug=False)
#         if debug:
#             print(f"\n\n🎭 starting Stage 1")

#         res = await dataset_routes.delete_partition_stage_1(full_auth=full_auth,
#                                                             dataset_id=dataset_id,
#                                                             dataset_partition_id=dataset_partition_id,
#                                                             debug=debug, session=session)
#         if debug:
#             print(f"\n\n🎭 Stage 1 response -- {res.status}")
#             print(res)

#         stage_2_res = None
#         if debug:
#             print('starting Stage 2')
#         stage_2_res = await dataset_routes.delete_partition_stage_2(full_auth=full_auth,
#                                                                     dataset_id=dataset_id,
#                                                                     dataset_partition_id=dataset_partition_id,
#                                                                     debug=debug, session=session)
#         if debug:
#             print(f"\n\n🎭 Stage 2 response -- {stage_2_res.status}")

#         stage_3_res = None
#         if debug:
#             print('starting Stage 3')
#         stage_3_res = await dataset_routes.index_dataset(full_auth=full_auth,
#                                                          dataset_id=dataset_id,
#                                                          debug=debug, session=session)
#         if debug:
#             print(f"\n\n🎭 Stage 3 response -- {stage_3_res.status}")

#         if is_close_session:
#             await session.close()

#         if debug:
#             print(stage_3_res)

#         if stage_3_res.status == 200:
#             return res.response

#     async def reset_dataset(self,
#                             full_auth: DomoFullAuth = None,
#                             is_index: bool = True,
#                             debug: bool = False
#                             ):
#         execute_reset = input(
#             "This function will delete all rows.  Type BLOW_ME_AWAY to execute:")

#         if execute_reset != 'BLOW_ME_AWAY':
#             print("You didn't type BLOW_ME_AWAY, moving on.")
#             return None

#         full_auth = full_auth or self.full_auth
#         dataset_id = self.id

#         if not full_auth:
#             raise Exception("full_auth required")

#         session = aiohttp.ClientSession()

#         # create empty dataset to retain schema
#         empty_df = await self.query_dataset_private(full_auth=full_auth,
#                                                     dataset_id=dataset_id,
#                                                     sql="SELECT * from table limit 1",
#                                                     session=session,
#                                                     debug=debug)
#         empty_df = empty_df.head(0)

#         # get partition list
# #         partition_list = await dataset_routes.list_partitions(full_auth=full_auth,
# #                                                               dataset_id=self.id,
# #                                                               debug=debug,
# #                                                               session=session)

# #         if len(partition_list) > 0:
# #             partition_list = chunk_list(partition_list, 100)

# #             for index, pl in enumerate(partition_list):
# #                 print(f'🥫 starting chunk {index + 1} of {len(partition_list)}')

# #                 await asyncio.gather(*[self.delete_partition(full_auth=full_auth,
# #                                                              dataset_partition_id=partition.get('partitionId'),
# #                                                              session=session,
# #                                                              empty_df=empty_df,
# #                                                              debug=False) for partition in pl])
# #                 if is_index:
# #                     await self.index_dataset(session=session)

#         res = await self.upload_csv(upload_df=empty_df,
#                                     upload_method='REPLACE',
#                                     is_index=is_index,
#                                     session=session,
#                                     debug=False)

#         await session.close()
#         return True

#     async def delete(self,
#                      dataset_id=None,
#                      full_auth: DomoFullAuth = None,
#                      debug: bool = False,
#                      session: aiohttp.ClientSession = None):
#         try:
#             is_close_session = False

#             if not session:
#                 session = aiohttp.ClientSession()
#                 is_close_session = True

#             return await dataset_routes.delete(
#                 full_auth=full_auth or self.full_auth,
#                 dataset_id=dataset_id or self.id,
#                 debug=debug,
#                 session=session)

#         finally:
#             if is_close_session:
#                 await session.close()

#     # async def create(self,
#     #                   ds_name,
#     #                   ds_type ='api',
#     #                   schema = { "columns": [ {
#     #                       "name": 'col1',
#     #                       "type": 'LONG',
#     #                       "metadata": None,
#     #                       "upsertKey": False}
#     #                   ]},
#     #                   full_auth:DomoFullAuth = None,
#     #                   debug:bool = False)